# Importando o Dataset

In [1]:
import json

In [2]:
teste = open('Dataset.txt', 'r')
dataset = json.load(teste)

# Chromadb - Armazenando o dataset

In [3]:
import sys
print(sys.executable)

/usr/bin/python


In [4]:

for i in dataset:
    a = f"""{{
      "date":"{i['date']}",
      "time":"{i['time']}",
      "period":"{i['period']}",
      "day_of_week":"{i['day_of_week']}",
      "season":"{i['season']}",
      "session":"{i['session']}"
      }}
      
      {i["description"]}
      """

    if (i['date'] == '2025-02-12'):
        print(a)

{
      "date":"2025-02-12",
      "time":"07:20:00",
      "period":"manhã",
      "day_of_week":"quarta-feira",
      "season":"verão",
      "session":"café-da-manhã"
      }
      
      Preparou café coado em filtro de papel, observando a primeira luz do dia refletir no líquido âmbar enquanto a chaleira assobiava. Um pão francês amanhecido foi transformado em torradas douradas, untadas com manteiga que derretia em veios translúcidos. Do quintal, vinha o barulho de pássaros disputando migalhas caídas da mesa externa, enquanto o cheiro de terra molhada da chuva noturna ainda pairava no ar.
      
{
      "date":"2025-02-12",
      "time":"12:50:00",
      "period":"meio-dia",
      "day_of_week":"quarta-feira",
      "season":"verão",
      "session":"almoço"
      }
      
      Reaqueceu arroz do dia anterior, adicionando ervilhas congeladas que estalavam na panela. Cortou tomates maduros em fatias desiguais, cujo suco se misturou ao azeite no prato formando poças rosadas. Durante

In [5]:
import ollama
import chromadb

In [ ]:
client = chromadb.PersistentClient(path="chroma/")
collection = client.get_or_create_collection(name="temporal")

ollama.pull("granite-embedding")

for d, i in enumerate(dataset):
    a = f"""{{
      "date":"{i['date']}",
      "time":"{i['time']}",
      "period":"{i['period']}",
      "day_of_week":"{i['day_of_week']}",
      "season":"{i['season']}",
      "session":"{i['session']}"
      }}
      
      {i["description"]}
      """
    embedding = ollama.embed(model="granite-embedding:278m", input=a)["embeddings"]

    collection.add(
        ids=[str(d)],
        embeddings=embedding,
        documents=[a]
    )

In [7]:
collection.count()

44

# LLM - Requisição Inicial

In [11]:
pergunta = "O que almocei no dia 2025-02-26?"

In [ ]:
embeddingPergunta = ollama.embed(model="granite-embedding:278m", input=pergunta)["embeddings"]
memorias = collection.query(query_embeddings=embeddingPergunta, n_results=6)["documents"]

In [13]:
memorias

[['{\n      "date":"2025-02-10",\n      "time":"22:10:00",\n      "period":"noite",\n      "day_of_week":"terça-feira",\n      "season":"verão",\n      "session":"pré-madrugada"\n      }\n      \n      Descobriu um easter egg em uma série de ficção científica que assistia: o código QR em um pôster de fundo levava a um curta-metragem secreto do diretor, cheio de referências à cultura cyberpunk dos anos 90.\n      ',
  '{\n      "date":"2025-03-25",\n      "time":"12:00:00",\n      "period":"meio-dia",\n      "day_of_week":"quarta-feira",\n      "season":"outono",\n      "session":"almoço"\n      }\n      \n      Descobriu um dente incrustado na azeitona da salada enquanto almoçava. O restaurante ofereceu sobremesa grátis, mas o garçom sussurrou: \'Isso acontece mais do que você imagina\', com um sorriso que revelou um canino faltando.\n      ',
  '{\n      "date":"2025-02-10",\n      "time":"14:35:00",\n      "period":"tarde",\n      "day_of_week":"terça-feira",\n      "season":"verão",